# OM Feedback Interaction

This notebook illustrates feedback system implemented by InSight. Unlike other systems, feedback system needs active API usage and this notebook created to help you how to use the API endpoints.

** Steps: **

1. Request contents specific to users.
  * News
  * Events
  * Questions
2. Send feedback to the requested and selected content.

## Scenarios

In this scenario, a system called **OutSight**, requests different types of contents for one of its user (id: 135574293). Then, sends the feedbacks of its user back to the **InSight** system.

### Scenario 1 - News

### Request Content

First, OutSight requests contents to show the user.

In [2]:
import requests

insightIP = 'http://178.62.229.16'
insightPort = '8484'
insightVersion = 'v1.1'

# parameters of news are crm_ids and api_key
crm_ids = [135574293]
api_key = "" #needs to be filled

insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
request = '/recommendation/news?' + 'crm_ids=' + str(crm_ids) + '&' + 'api_key=' + api_key

# send a request
res = requests.get(insightSetting + request)

Recommendation News API endpoint returns **crm_id**, **twitter_id** of the given user, **feedback_type** of the requested content and **all recommended news** for given user. Below, you can find some useful information of 3 recommended news for the user with id: 135574293.

In [5]:
res.json()['users'][0].keys()

dict_keys(['crm_id', 'news', 'twitter_id', 'feedback_type'])

In [12]:
print("crm_id: " + res.json()['users'][0]['crm_id'])
print("twitter_id: " + res.json()['users'][0]['twitter_id'])
print("feedback_type: " + res.json()['users'][0]['feedback_type'])
print()

crm_id: 135574293
twitter_id: 323040062
feedback_type: rate_1_to_5



In [9]:
res.json()['users'][0]['news'][0].keys()

dict_keys(['temp_id', 'link_id', 'title', 'full_text', 'popularity', 'domain', 'source', 'published_at', 'summary', 'im', 'keywords', 'url'])

(Attention: returned object includes more information (above), but for simplicity we only printed some (below))

In [15]:
for news in res.json()['users'][0]['news']:
    print("temp_id: " + str(news['temp_id']))
    print("url: " + news['url'])
    print("summary: " + news['summary'])
    print()

temp_id: 4
url: https://techxplore.com/news/2018-03-raspberry-pi-latest-member-price.html
summary: Say hello to the new Raspberry Pi 3 Model B+.
"This latest iteration has the same footprint as both the Raspberry Pi 2 Model B and Raspberry Pi 3 Model B, which means it's about the size of a deck of cards," said Darrell Etherington in TechCrunch.
It's a new version of the Raspberry Pi Model B, and an applauded upgrade by many.
Raspberry Pi 3 Model B+ is available now for the same $35 price.
Upton said, all in all, that, with the Raspberry Pi 3 Model B+, they rolled up fixes and came up with a neater, tidier-looking board.

temp_id: 5
url: http://health-miracle.com/big-data-suggests-big-potential-for-urban-farming/
summary: Taken together, these fringe benefits make metropolitan farming worth as much as$160 billion every year around the world.
”“ This is the very first worldwide price quote of the capacity for metropolitan farming, ” Costanza composed in an e-mail.
”Meanwhile, policy in t

### Send Feedback

After receiveng news and feedback_type from the API endpoint, OutSight can now show multiple news or just one specific news to the user with the proper feedback input according to the feedback_type.

The feedback_type of the news is stated as **rate_1_to_5** which means that it requires an integer from 1 to 5 where 1 corresponds to bad and 5 corresponds the good.

To send the feedback of the user to the InSight system OutSight must specify 3 things:
* **crm_id** of the user which gives feedback
* **feedback** according to the feedback_type
* **temp_id** of the content.

**temp_id** is an id that is assigned to each content daily when they are created. As it is name suggests, it is a **temporary id**. A content obtains a **permanent id** when a feedback is sent to the system for this content. The real permanent **content_id** can be obtained after the feedback is sent where you can find the process below.

**temp_id**: 4

**url:** https://techxplore.com/news/2018-03-raspberry-pi-latest-member-price.html

**summary:** Say hello to the new Raspberry Pi 3 Model B+.
"This latest iteration has the same footprint as both the Raspberry Pi 2 Model B and Raspberry Pi 3 Model B, which means it's about the size of a deck of cards," said Darrell Etherington in TechCrunch.
It's a new version of the Raspberry Pi Model B, and an applauded upgrade by many.
Raspberry Pi 3 Model B+ is available now for the same $35 price.
Upton said, all in all, that, with the Raspberry Pi 3 Model B+, they rolled up fixes and came up with a neater, tidier-looking board.

In [40]:
import requests

insightIP = 'http://178.62.229.16'
insightPort = '8484'
insightVersion = 'v1.1'

# parameters of send_feedback are crm_id, temp_id, feedback and api_key
crm_id = 135574293
temp_id = 4
feedback = 5
api_key = "" #needs to be filled

insightSetting = insightIP + ':' + insightPort + '/api/' + insightVersion 
request = '/feedback/send_feedback?'+'crm_id='+str(crm_id)+'&temp_id='+str(temp_id)+'&feedback='+str(feedback)+'&api_key='+api_key

# send a request
res = requests.get(insightSetting + request)

OutSight chose the content with **temp_id** with **4** to show the user and the user gave **5** as a feedback.

In [41]:
res.json()

{'DateTime': '2018-04-10 13:07:11',
 'content_id': 25,
 'crm_id': '135574293',
 'feedback': '5',
 'feedback_id': 27,
 'feedback_type': 'rate_1_to_5'}

As you can see above, when the **send feedback** API endpoint is succesful, it returns very useful information including the permanent **content_id** of the sent content.

For example, if the same content is sent with a different temp_id, InSight API recognizes this content and returns the same permanent **content_id**.

(!Attention: send_feedback API always works with **temp_id**. It only return **content_id** to give information back to the caller)